In [1]:
#|echo: false
import pandas as pd, numpy as np, matplotlib.pyplot as plt, altair as alt, pytz
from fastcore.all import *
from datetime import datetime, timedelta

## Get data

In [2]:
#|echo: false
print(f'Last execution time: {datetime.now(pytz.timezone("America/Lima")).strftime("%d/%m/%Y %T")}')

Last execution time: 26/08/2024 06:31:30


In [3]:
#|code-summary: Products type filter
explore_types = ['frutas', 'lacteos', 'verduras', 'embutidos', 'panaderia', 'desayuno', 'congelados', 'abarrotes',
                 'aves', 'carnes', 'pescados']

In [4]:
#|code-summary: Data table
path = Path('../../output')
csv_files = L(path.glob('*.csv')).filter(lambda o: os.stat(o).st_size>0)
pat_store = re.compile('(.+)\_\d+')
pat_date = re.compile('.+\_(\d+)')
df = (
    pd.concat([pd.read_csv(o).assign(store=pat_store.match(o.stem)[1], date=pat_date.match(o.stem)[1])
               for o in csv_files], ignore_index=True)
    .pipe(lambda d: d.assign(
        name=d.name.str.lower()+' ('+d.store+')',
        sku=d.id.where(d.sku.isna(), d.sku).astype(int),
        date=pd.to_datetime(d.date)
    ))
    .drop('id', axis=1)
    .loc[lambda d: d.category.str.contains('|'.join(explore_types))]
    # Filter products with recent data
#     .loc[lambda d: d.name.isin(d.groupby('name').date.max().loc[ge(datetime.now()-timedelta(days=30))].index)]
    # Filter empty prices
    .loc[lambda d: d.price>0]
)
print(df.shape)
df.sample(3)

(1018744, 8)


,sku,name,brand,category,uri,price,store,date
1834362,10021755,panela granulada orgánica bell's caja 50 sobre...,BELL'S,https://www.plazavea.com.pe/abarrotes,https://www.plazavea.com.pe/panela-granulada-o...,13.2,plaza_vea,2022-09-12
493227,10906303,leche sin lactosa gloria lata 395g (plaza_vea),GLORIA,https://www.plazavea.com.pe/lacteos-y-huevos,https://www.plazavea.com.pe/leche-sin-lactosa-...,4.4,plaza_vea,2024-02-29
2102370,1828,gomas dulces 2 cerritos sabor a eucalipto bols...,2 CERRITOS,https://www.plazavea.com.pe/abarrotes,https://www.plazavea.com.pe/gomas-dulces-2-cer...,10.9,plaza_vea,2024-07-22


## Top changes (ratio)

In [5]:
top_changes = (df
 # Use last 30 days of data to compare prices
 .loc[lambda d: d.date>=(datetime.now()-timedelta(days=30))]
 .sort_values('date')
 # Get percentage change
 .assign(change=lambda d: d
     .groupby(['store','sku'], as_index=False)
     .price.transform(lambda d: (d-d.shift())/d.shift())
 )
 .groupby(['store','sku'], as_index=False)
 .agg({'price':'last', 'change':'mean', 'date':'last'})
 .rename({'price':'last_price', 'date':'last_date'}, axis=1)
 .dropna()
 .loc[lambda d: d.last_date==d.last_date.max()]
 .loc[lambda d: d.change.abs().sort_values(ascending=False).index]
)
top_changes.head(3)

,store,sku,last_price,change,last_date
390,plaza_vea,1521,14.40,0.172234,2024-08-26
258,plaza_vea,1175,15.89,0.129197,2024-08-26
1364,plaza_vea,10808,7.99,-0.126904,2024-08-26


In [6]:
def plot_changes(df_changes, title):
    selection = alt.selection_point(fields=['name'], bind='legend')
    dff = df_changes.drop('change', axis=1).merge(df, on=['store','sku'])
    return (dff
     .pipe(alt.Chart)
     .mark_line(point=True)
     .encode(
         x='date',
         y='price',
         color=alt.Color('name').scale(domain=sorted(dff.name.unique().tolist())),
         tooltip=['name','price','last_price']
     )
     .add_params(selection)
     .transform_filter(selection)
     .interactive()
     .properties(width=650, title=title)
     .configure_legend(orient='top', columns=3)
    )

In [7]:
top_changes.head(10).pipe(plot_changes, 'Top changes')

alt.Chart(...)

In [8]:
(top_changes
 .sort_values('change')
 .head(10)
 .pipe(plot_changes, 'Top drops')
)

alt.Chart(...)

In [9]:
(top_changes
 .sort_values('change')
 .tail(10)
 .pipe(plot_changes, 'Top increases')
)

alt.Chart(...)

## Top changes (absolute values)

In [10]:
top_changes_abs = (df
 # Use last 30 days of data to compare prices
 .loc[lambda d: d.date>=(datetime.now()-timedelta(days=30))]
 .sort_values('date')
 # Get percentage change
 .assign(change=lambda d: d
     .groupby(['store','sku'], as_index=False)
     .price.transform(lambda d: (d-d.shift()).iloc[-1])
 )
 .groupby(['store','sku'], as_index=False)
 .agg({'price':'last', 'change':'mean', 'date':'last'})
 .rename({'price':'last_price', 'date':'last_date'}, axis=1)
 .dropna()
 .loc[lambda d: d.last_date==d.last_date.max()]
 .loc[lambda d: d.change.abs().sort_values(ascending=False).index]
)
top_changes_abs.head(3)

,store,sku,last_price,change,last_date
1820,plaza_vea,25385,69.89,-22.1,2024-08-26
5719,plaza_vea,11434464,29.50,-20.4,2024-08-26
1944,plaza_vea,29088,96.90,-20.0,2024-08-26


In [11]:
top_changes_abs.head(10).pipe(plot_changes, 'Top changes')

alt.Chart(...)

In [12]:
(top_changes_abs
 .sort_values('change')
 .head(10)
 .pipe(plot_changes, 'Top drops')
)

alt.Chart(...)

In [13]:
(top_changes_abs
 .sort_values('change')
 .tail(10)
 .pipe(plot_changes, 'Top increases')
)

alt.Chart(...)

## Search specific products

In [14]:
#|echo: false
#|output: false
names = df.name[df.name.str.contains(r'(?=.*pollo)(?=.*entero).*') &
                ~df.name.str.contains(r'marinado|aderezo')].unique().tolist()
names

['pollo entero congelado perdix bolsa 1400g (plaza_vea)',
 'pollo entero\xa0artisan\xa0libre de antibióticos x kg (plaza_vea)',
 'pollo entero fresco con menudencia (plaza_vea)',
 'pollo entero fresco metro x kg (metro)',
 'pollo entero sin menudencia x kg (metro)',
 'pollo entero light  x kg (metro)',
 'pollo entero sin menudencia importado x kg (metro)',
 'pollo entero con menudencia x kg (wong)',
 'pollo entero sin menudencia x kg (wong)',
 'pollo entero light  x kg (wong)',
 'pollo entero sin menudencia congelado perdix 1300g (plaza_vea)']

In [15]:
(df
 .loc[df.name.isin(names)]
 .pipe(alt.Chart)
 .mark_line(point=True)
 .encode(x='date', y='price', color='name', tooltip=['name','price'])
 .properties(width=650, title='Pollo')
 .interactive()
 .configure_legend(orient='top', columns=3)
)

alt.Chart(...)

In [16]:
#|echo: false
#|output: false
names = df.name[df.name.str.contains(r'palta') &
                ~df.name.str.contains(r'shampoo|humectante|vino|salsa|acondicionador|aceite')].unique().tolist()
names

["palta fuerte bell's madura (plaza_vea)",
 'palta fuerte (plaza_vea)',
 'palta hass madura la caserita empaque 500g (plaza_vea)',
 'palta hass (plaza_vea)',
 'palta fuerte metro x kg (metro)',
 'palta hass natifrut x kg (metro)',
 'palta madura cremosita x kg (metro)',
 'palta hass x kg (metro)',
 'palta nava metro x kg (metro)',
 'palta nava x kg (wong)',
 'palta fuerte verde x kg (wong)',
 'palta hass natifrut x kg (wong)',
 'palta hass orgánica 1kg (wong)',
 'palta fuerte madura empacada x kg (wong)',
 'palta naval x kg (plaza_vea)',
 'palta super fuerte x kg (metro)',
 'palta hass madura natifrut 500g (metro)',
 'palta fuerte x kg (plaza_vea)',
 'dip con palta casa verde frasco 225g (plaza_vea)',
 'palta fuerte malla 1kg z (plaza_vea)']

In [17]:
(df
 .loc[df.name.isin(names)]
 .pipe(alt.Chart)
 .mark_line(point=True)
 .encode(x='date', y='price', color='name', tooltip=['name','price'])
 .properties(width=650, title='Palta')
 .interactive()
 .configure_legend(orient='top', columns=3)
)

alt.Chart(...)

In [18]:
#|echo: false
#|output: false
names = df.name[df.name.str.contains(r'(?=.*aceite)(?=.*vegetal)(?=.*900).*') &
                ~df.name.str.contains(r'atun|atún|pack|filete|caballa|tacos|sardinas')].unique().tolist()
names

['aceite vegetal cil botella 900ml (plaza_vea)',
 'aceite vegetal cocinero botella 900ml (plaza_vea)',
 'aceite vegetal nicolini botella 900ml (plaza_vea)',
 'aceite vegetal primor premium botella 900ml (plaza_vea)',
 'aceite vegetal deleite botella 900ml (plaza_vea)',
 'aceite vegetal primor clásico botella 900ml (plaza_vea)',
 "aceite vegetal bell's botella 900ml (plaza_vea)",
 'aceite vegetal deleite premium botella 900ml (plaza_vea)',
 'aceite vegetal del cielo botella 900ml (plaza_vea)',
 'aceite vegetal nicolini 900ml (metro)',
 'aceite vegetal primor premium 900ml (metro)',
 'aceite vegetal metro 900ml (metro)',
 'aceite vegetal deleite premium 900ml (metro)',
 'aceite vegetal máxima 900ml (metro)',
 'aceite vegetal cocinero 900ml (metro)',
 'aceite vegetal primor corazón 900ml (metro)',
 'aceite vegetal primor clásico 900ml (metro)',
 'aceite vegetal primor corazón 900ml (wong)',
 'aceite vegetal primor premium 900ml (wong)',
 'aceite vegetal primor clásico 900ml (wong)',
 'ace

In [19]:
(df
 .loc[df.name.isin(names)]
 .pipe(alt.Chart)
 .mark_line(point=True)
 .encode(x='date', y='price', color='name', tooltip=['name','price'])
 .properties(width=650, title='Aceite')
 .interactive()
 .configure_legend(orient='top', columns=3)
)

alt.Chart(...)

In [20]:
#|echo: false
#|output: false
names = df.name[
    df.name.str.contains(r'(?=.*harina)(?=.*1kg).*')
#     & ~df.name.str.contains(r'atun|atún|pack|filete|caballa|tacos|sardinas')
].unique().tolist()
names

['harina de maíz amarillo p.a.n. precocida bolsa 1kg (plaza_vea)',
 'harina de trigo preparada blanca flor bolsa 1kg (plaza_vea)',
 'harina integral grano de oro bolsa 1kg (plaza_vea)',
 'harina de trigo nicolini sin preparar bolsa 1kg (plaza_vea)',
 'harina de maíz blanco p.a.n. precocida bolsa 1kg (plaza_vea)',
 'harina la casa marimel integral bolsa 1kg (plaza_vea)',
 'harina sin preparar favorita paquete 1kg (plaza_vea)',
 'harina preparada favorita paquete 1kg (plaza_vea)',
 'harina preparada grano de oro paquete 1kg (plaza_vea)',
 'harina selecta molitalia bolsa 1kg (plaza_vea)',
 'harina de trigo nicolini preparada bolsa 1kg (plaza_vea)',
 'harina sin preparar blanca flor bolsa 1kg (plaza_vea)',
 "harina de trigo sin preparar bell's bolsa 1kg (plaza_vea)",
 "harina bell's de trigo preparada bolsa 1kg (plaza_vea)",
 'harina molitalia preparada bolsa 1kg (plaza_vea)',
 'harina sin preparar grano de oro bolsa 1kg (plaza_vea)',
 'harina de trigo sin preparar molitalia sin preparar b

In [21]:
(df
 .loc[df.name.isin(names)]
 .pipe(alt.Chart)
 .mark_line(point=True)
 .encode(x='date', y='price', color='name', tooltip=['name','price'])
 .properties(width=650, title='Aceite')
 .interactive()
 .configure_legend(orient='top', columns=3)
)

alt.Chart(...)

In [22]:
#|echo: false
#|output: false
names = df.name[
    df.name.str.contains(r'(?=.*limón).*(?=.*kg).*')
#     df.name.str.contains(r'limón')
#     & ~df.name.str.contains(r'atun|atún|pack|filete|caballa|tacos|sardinas')
].unique().tolist()
names

['limón x kg (metro)',
 'limón tahiti metro x kg (metro)',
 'limón x kg (wong)',
 'limón tahití x kg (plaza_vea)',
 'limón dulce x kg (wong)',
 'limón tahíti x kg (wong)']

In [23]:
(df
 .loc[df.name.isin(names)]
 .pipe(alt.Chart)
 .mark_line(point=True)
 .encode(x='date', y='price', color='name', tooltip=['name','price'])
 .properties(width=650, title='Aceite')
 .interactive()
 .configure_legend(orient='top', columns=3)
)

alt.Chart(...)